##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Introduction to TensorFlow
# Notebook - Workflow

*Disclosure: This notebook is an adaptation of official TensorFlow tutorials.*

This notebook is covering multiple aspects of common Deep Learning workflows:

*   Save and load Keras models
*   Dataset Management
*   AutGraph in TensorFlow 2.0

More information of each introduced topic can be found on the TensorFlow website.

# Save and load Keras models

Model progress can be saved during—and after—training. This means a model can resume where it left off and avoid long training times. Saving also means you can share your model and others can recreate your work. When publishing research models and techniques, most machine learning practitioners share:

* code to create the model, and
* the trained weights, or parameters, for the model

Sharing this data helps others understand how the model works and try it themselves with new data.

Caution: Be careful with untrusted code—TensorFlow models are code. See [Using TensorFlow Securely](https://github.com/tensorflow/tensorflow/blob/master/SECURITY.md) for details.

### Options

There are different ways to save TensorFlow models—depending on the API you're using. This guide uses [tf.keras](https://www.tensorflow.org/guide/keras), a high-level API to build and train models in TensorFlow. For other approaches, see the TensorFlow  [Save and Restore](https://www.tensorflow.org/guide/saved_model) guide or [Saving in eager](https://www.tensorflow.org/guide/eager#object-based_saving).

## Setup

### Installs and imports
Install and import TensorFlow and dependencies:

In [0]:
!pip install pyyaml h5py  # Package dependencies to save models in HDF5 format

In [0]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

### Get an example dataset

To demonstrate how to save and load weights, you'll use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). To speed up these runs, use the first 2000 examples:

In [0]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:2000]
test_labels = test_labels[:2000]

train_images = train_images[:2000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:2000].reshape(-1, 28 * 28) / 255.0

### Define a model

Start by building a simple sequential model:

In [0]:
# Define a simple sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

## Save checkpoints during training

You can use a trained model without having to retrain it, or pick-up training where you left off—in case the training process was interrupted. The `tf.keras.callbacks.ModelCheckpoint` callback allows to continually save the model both *during* and at *the end* of training.

### Checkpoint callback usage

Create a `tf.keras.callbacks.ModelCheckpoint` callback that saves weights only during training:

In [0]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

This creates a single collection of TensorFlow checkpoint files that are updated at the end of each epoch:

In [0]:
!ls {checkpoint_dir}

Create a new, untrained model. When restoring a model from weights-only, you must have a model with the same architecture as the original model. Since it's the same model architecture, you can share weights despite that it's a different *instance* of the model.

Now rebuild a fresh, untrained model, and evaluate it on the test set. An untrained model will perform at chance levels (~10% accuracy):

In [0]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

Then load the weights from the checkpoint and re-evaluate:

In [0]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

### Checkpoint callback options

The callback provides several options to provide unique names for checkpoints and adjust the checkpointing frequency.

Train a new model, and save uniquely named checkpoints once every five epochs:

In [0]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    # save_best_only=True  # you might also consider saving only the best model
    save_freq='epoch')  # save a checkpoint every epoch.

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
          callbacks=[cp_callback],
          validation_data=(test_images,test_labels),
          verbose=0)

Now, look at the resulting checkpoints and choose the latest one:

In [0]:
!ls {checkpoint_dir}

In [0]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

Note: the default tensorflow format only saves the 5 most recent checkpoints.

To test, reset the model and load the latest checkpoint:

In [0]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

## What are these files?

The above code stores the weights to a collection of [checkpoint](https://www.tensorflow.org/guide/saved_model#save_and_restore_variables)-formatted files that contain only the trained weights in a binary format. Checkpoints contain:
* One or more shards that contain your model's weights.
* An index file that indicates which weights are stored in a which shard.

If you are only training a model on a single machine, you'll have one shard with the suffix: `.data-00000-of-00001`

## Manually save weights

You saw how to load the weights into a model. Manually saving them is just as simple with the `Model.save_weights` method. By default, `tf.keras`—and `save_weights` in particular—uses the TensorFlow [checkpoint](../../guide/checkpoint.ipynb) format with a `.ckpt` extension (saving in [HDF5](https://js.tensorflow.org/tutorials/import-keras.html) with a `.h5` extension is covered in the [Save and serialize models](../../guide/keras/save_and_serialize#weights-only_saving_in_savedmodel_format) guide):

In [0]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

Note, that the above code might produce warnings. This happens, if the checkpoint contains parameter values, that are not used by the instantiated model or if not all variables of the instantiated model have a corresponding parameter within the checkpoint. Often times, this is the case for internal parameter values of the optimizer.

## Save the entire model

Call [`model.save`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#save) to save a model's architecture, weights, and training configuration in a single file/folder. This allows you to export a model so it can be used without access to the original Python code*. Since the optimizer-state is recovered, you can resume training from exactly where you left off.

Entire model can be saved in two different file formats (`SavedModel` and `HDF5`). It is to be noted that TensorFlow `SavedModel` format is the default file format in TF2.x. However, model can be saved in `HDF5` format. More details on saving entire model in the two file formats is described below.

Saving a fully-functional model is very useful—you can load them in TensorFlow.js ([Saved Model](https://www.tensorflow.org/js/tutorials/conversion/import_saved_model), [HDF5](https://www.tensorflow.org/js/tutorials/conversion/import_keras)) and then train and run them in web browsers, or convert them to run on mobile devices using TensorFlow Lite ([Saved Model](https://www.tensorflow.org/lite/convert/python_api#converting_a_savedmodel_), [HDF5](https://www.tensorflow.org/lite/convert/python_api#converting_a_keras_model_))

\*Custom objects (e.g. subclassed models or layers) require special attention when saving and loading. See the **Saving custom objects** section below 

### SavedModel format

The SavedModel format is another way to serialize models. Models saved in this format can be restored using `tf.keras.models.load_model` and are compatible with TensorFlow Serving. The [SavedModel guide](https://www.tensorflow.org/guide/saved_model) goes into detail about how to serve/inspect the SavedModel. The section below illustrates the steps to saving and restoring the model.

In [0]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model') 

The SavedModel format is a directory containing a protobuf binary and a Tensorflow checkpoint. Inspect the saved model directory:

In [0]:
# my_model directory
!ls saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
!ls saved_model/my_model

Reload a fresh Keras model from the saved model:

In [0]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

The restored model is compiled with the same arguments as the original model. Try running evaluate and predict with the loaded model:

In [0]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

### HDF5 format

Keras provides a basic save format using the [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) standard. 

In [0]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5') 

Now, recreate the model from that file:

In [0]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Check its accuracy:

In [0]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

Keras saves models by inspecting the architecture. This technique saves everything:

* The weight values
* The model's architecture
* The model's training configuration(what you passed to compile)
* The optimizer and its state, if any (this enables you to restart training where you left)

Keras is not able to save the `v1.x` optimizers (from `tf.compat.v1.train`) since they aren't compatible with checkpoints. For v1.x optimizers, you need to re-compile the model after loading—losing the state of the optimizer.


### Saving custom objects

If you are using the SavedModel format, you can skip this section. The key difference between HDF5 and SavedModel is that HDF5 uses object configs to save the model architecture, while SavedModel saves the execution graph. Thus, SavedModels are able to save custom objects like subclassed models and custom layers without requiring the orginal code.

To save custom objects to HDF5, you must do the following:

1. Define a `get_config` method in your object, and optionally a `from_config` classmethod.
  * `get_config(self)` returns a JSON-serializable dictionary of parameters needed to recreate the object.
  * `from_config(cls, config)` uses the returned config from `get_config` to create a new object. By default, this function will use the config as initialization kwargs (`return cls(**config)`).
2. Pass the object to the `custom_objects` argument when loading the model. The argument must be a dictionary mapping the string class name to the Python class. E.g. `tf.keras.models.load_model(path, custom_objects={'CustomLayer': CustomLayer})`

See the [Writing layers and models from scratch](https://www.tensorflow.org/guide/keras/custom_layers_and_models) tutorial for examples of custom objects and `get_config`.


# Dataset Management

The following chapter covers **Dataset API** and **TensorFlow Datasets**.

In [0]:
import tensorflow as tf

##Dataset API
TensorFlow provides a rich `tf.data.Dataset` API which allows you to easily manage complex pipeline operations.

The Dataset object is a Python iterable. This makes it possible to consume its elements using a for loop.


In [0]:
dataset = tf.data.Dataset.range(10)
for elem in dataset:
  print(elem.numpy())  # prints 0, 1, ..., 9

###Dataset manipulation

Datasets provide methods, that allow the manipulation of elements. As such the elements can be subject to arbitrary transformations.

In [0]:
dummy_data = tf.random.uniform([4])
print('Dummy data: ', dummy_data.numpy())

dataset1 = tf.data.Dataset.from_tensor_slices(tf.random.uniform([4]))
dataset1 = dataset1.map(lambda x: x*x)

dataset2 = tf.data.Dataset.from_tensor_slices(tf.random.uniform([4]))
dataset2 = dataset2.filter(lambda x: x > 0.5)

dataset3 = tf.data.Dataset.zip((dataset1, dataset2))
dataset3 = dataset3.map(lambda x, y: x+y)

print("\n dataset 1\n")
for elem in dataset1:
  print(elem.numpy())

print("\n dataset 2\n")
for elem in dataset2:
  print(elem.numpy())
  
print("\n dataset 3\n")
for elem in dataset3:
  print(elem.numpy())

------

##TensorFlow Datasets

TensorFlow Datasets provides a collection of datasets ready to use with TensorFlow. It handles downloading and preparing the data and constructing a [`tf.data.Dataset`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset).

###Installation

`pip install tensorflow-datasets`

Note that `tensorflow-datasets` expects you to have TensorFlow already installed, and currently depends on `tensorflow` (or `tensorflow-gpu`) >= `1.13.0`.

In [0]:
!pip install tensorflow-datasets

###Import required modules


In [0]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

###List the available datasets

Each dataset is implemented as a [`tfds.core.DatasetBuilder`](https://www.tensorflow.org/datasets/api_docs/python/tfds/core/DatasetBuilder) and you can list all available builders with `tfds.list_builders()`.

You can see all the datasets with additional documentation on the [datasets documentation page](https://github.com/tensorflow/datasets/blob/master/docs/datasets.md).

In [0]:
tfds.list_builders()

###`tfds.load`: A dataset in one line

[`tfds.load`](https://www.tensorflow.org/datasets/api_docs/python/tfds/load) is a convenience method that's the simplest way to build and load and `tf.data.Dataset`. Below, we load the MNIST training data.

In [0]:
ds, info = tfds.load(name="mnist", with_info=True)  # also fetch dataset info
mnist_train = ds["train"]

**Note:** The dataset can also be called directly by passing a split argument, i.e.  
 `mnist_train = tfds.load(name="mnist", split=tfds.Split.TRAIN)`

`tfds` returns a dictionary containing a "train" and a "test" `tf.data.Dataset` with respective keys.  
`info` contains furtherl information on the fetched dataset.


In [0]:
print("tfds type: ", type(ds), "\n")

print("tfds dictionary: ", ds, "\n")

print("train dataset type: ", type(mnist_train), "\n")

print("tfds info: ", info)


### Feature dictionaries

All `tfds` datasets contain feature dictionaries mapping feature names to Tensor values. A typical dataset, like MNIST, will have 2 keys: `"image"` and `"label"`. Below we inspect a single example.

In [0]:
mnist_example, = mnist_train.take(1)
image, label = mnist_example["image"], mnist_example["label"]

plt.imshow(image.numpy()[:, :, 0].astype(np.float32), cmap=plt.get_cmap("gray"))
print(f"Label: {label.numpy()}")

### DatasetBuilder

`tfds.load` is really a thin conveninence wrapper around `DatasetBuilder`. We can accomplish the same as above directly with the MNIST `DatasetBuilder`.

In [0]:
mnist_builder = tfds.builder("mnist")
mnist_builder.download_and_prepare()
mnist_train = mnist_builder.as_dataset(split=tfds.Split.TRAIN)
mnist_train

### DatasetInfo

After generation, the builder contains useful information on the dataset:

In [0]:
info = mnist_builder.info
print(info)

`DatasetInfo` also contains useful information about the features:

In [0]:
print(info.features)
print(info.features["label"].num_classes)
print(info.features["label"].names)

You can also load the `DatasetInfo` directly with `tfds.load` using `with_info=True`.

In [0]:
dataset, info = tfds.load("mnist", split="test", with_info=True)
print(info)

### Input pipelines

Once you have a `tf.data.Dataset` object, it's simple to define the rest of an input pipeline suitable for model training by using the [`tf.data` API](https://www.tensorflow.org/guide/datasets).

Here we'll repeat the dataset so that we have an infinite stream of shuffled examples. After the augmentation batches of 32 are created.

In [0]:
mnist_train_shuffled = mnist_train.repeat().shuffle(1024)

### Augmentation

The dataset API makes it easy to integrate mapping functions. As such, pre-processing steps (e.g. data parsing, augmentation, ...) can be included within a dataset.map() call to apply transformations on the fly.

Add an augmentation transformation to the dataset pipeline.


In [0]:
def augmentation(input, HEIGHT=28, WIDTH=28, NUM_CHANNELS=1):
    print(input)
    x = tf.image.per_image_standardization(input['image'])  # normalize image
    x = tf.image.resize_with_crop_or_pad(x, HEIGHT + 8, WIDTH + 8)  # resize image
    x = tf.image.random_crop(x, [HEIGHT, WIDTH, NUM_CHANNELS])  # random crop
    x = tf.image.random_flip_left_right(x)  # random flip
    # Be careful to select only those operations, that apply to your dataset.
    # Flipping might not be a sensible choice for a MNIST classification
    return {'image': x, 'label': input['label']}
  
mnist_train_aug = mnist_train_shuffled.map(augmentation)

In [0]:
# prefetch will enable the input pipeline to asynchronously fetch batches while
# your model is training.
mnist_train_batched = mnist_train_aug.batch(32).prefetch(tf.data.experimental.AUTOTUNE)

# Now you could loop over batches of the dataset and train
# for batch in mnist_train:
#   ...

In [0]:
mnist_example, = mnist_train_batched.take(1)
image, label = mnist_example["image"], mnist_example["label"]

plt.imshow(image.numpy()[0, :, :, 0].astype(np.float32), cmap=plt.get_cmap("gray"))
print(f"Label: {label.numpy()}")

# AutoGraph in TensorFlow 2.0

TF 2.0 brings together the ease of eager execution and the power of TF 1.0. At the center of this merger is `tf.function`, which allows you to transform a subset of Python syntax into portable, high-performance TensorFlow graphs.

A cool new feature of `tf.function` is AutoGraph, which lets you write graph code using natural Python syntax. For a list of the Python features that you can use with AutoGraph, see [AutoGraph Capabilities and Limitations](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/LIMITATIONS.md). For more details about `tf.function`, see the [TF 2.0: Functions, not Sessions](https://github.com/tensorflow/community/blob/master/rfcs/20180918-functions-not-sessions-20.md). For more details about AutoGraph, see `tf.autograph`.

This tutorial will walk you through the basic features of `tf.function` and AutoGraph.

## Setup

Import TensorFlow

In [0]:
!pip install tensorflow
import tensorflow as tf
import numpy as np

## The `tf.function` decorator

When you annotate a function with `tf.function`, you can still call it like any other function. But it will be compiled into a graph, which means you get the benefits of faster execution, running on GPU or TPU, or exporting to SavedModel.

In [0]:
@tf.function
def simple_nn_layer(x, y):
  return tf.nn.relu(tf.matmul(x, y))


x = tf.random.uniform((3, 3))
y = tf.random.uniform((3, 3))

simple_nn_layer(x, y)

If we examine the result of the annotation, we can see that it's a special callable that handles all interactions with the TensorFlow runtime.

In [0]:
simple_nn_layer

If your code uses multiple functions, you don't need to annotate them all - any functions called from an annotated function will also run in graph mode.

In [0]:
def linear_layer(x):
  return 2 * x + 1


@tf.function
def deep_net(x):
  return tf.nn.relu(linear_layer(x))


deep_net(tf.constant((1, 2, 3)))

Functions can be faster than eager code, for graphs with many small ops. The performance difference will increase with the number of ops included. But for graphs with a few expensive ops (like convolutions), you may not see much speedup. 


In [0]:
# import timeit to measure the execution time
import timeit

In [0]:

conv_layer = tf.keras.layers.Conv2D(100, 3)

@tf.function
def conv_fn(image):
  return conv_layer(image)

image = tf.zeros([1, 200, 200, 100])
# warm up
conv_layer(image); conv_fn(image)
print("Eager conv:", timeit.timeit(lambda: conv_layer(image), number=10))
print("Function conv:", timeit.timeit(lambda: conv_fn(image), number=10))

In [0]:
lstm_cell = tf.keras.layers.LSTMCell(10)

@tf.function
def lstm_fn(input, state):
  return lstm_cell(input, state)

input_lstm = tf.zeros([10, 10])
state_lstm = [tf.zeros([10, 10])] * 2
# warm up
lstm_cell(input_lstm, state_lstm); lstm_fn(input_lstm, state_lstm)
print("Eager lstm:", timeit.timeit(lambda: lstm_cell(input_lstm, state_lstm), number=10))
print("Function lstm:", timeit.timeit(lambda: lstm_fn(input_lstm, state_lstm), number=10))

## Use Python control flow

When using data-dependent control flow inside `tf.function`, you can use Python control flow statements and AutoGraph will convert them into appropriate TensorFlow ops. For example, `if` statements will be converted into `tf.cond()` if they depend on a `Tensor`.

In the example below, `x` is a `Tensor` but the `if` statement works as expected:

In [0]:
@tf.function
def square_if_positive(x):
  if x > 0:
    x = x * x
  else:
    x = 0
  return x


print(f'square_if_positive(2) = {square_if_positive(tf.constant(2))}')
print(f'square_if_positive(-2) = {square_if_positive(tf.constant(-2))}')

AutoGraph supports common Python statements like `while`, `for`, `if`, `break`, `continue` and `return`, with support for nesting. That means you can use `Tensor` expressions in the condition of `while` and `if` statements, or iterate over a `Tensor` in a `for` loop.

In [0]:
@tf.function
def sum_even_fn(items):
  s = 0
  for c in items:
    if c % 2 > 0:
      continue
    s += c
  return s

sum_even_fn(tf.constant(tf.range(500)))

AutoGraph also provides a low-level API for advanced users. For example we can use it to have a look at the generated code.

In [0]:
print(tf.autograph.to_code(sum_even_fn.python_function))

Let's compare this control flow for Python vs TensorFlow Graph code. For simple functions, it might not be a good idea to introduce a computation graph overhead.

In [0]:
def sum_even(items):
  s = 0
  for c in items:
    if c % 2 > 0:
      continue
    s += c
  return s


sum_even(range(500))

# warm up
in_sum = list(range(500)); sum_even(in_sum)
in_sum_fn = tf.constant(tf.range(500)); sum_even_fn(in_sum_fn)
print(f"Results: Python code: {sum_even(in_sum)}, TF code: {sum_even_fn(in_sum_fn).numpy()}")
print("Python:  ", timeit.timeit(lambda: sum_even(in_sum), number=10))
print("TF graph:", timeit.timeit(lambda: sum_even_fn(in_sum_fn), number=10))

## Keras and AutoGraph

AutoGraph is available by default in non-dynamic Keras models. For more information, see `tf.keras`.

In [0]:
class CustomModel(tf.keras.models.Model):

  @tf.function
  def call(self, input_data):
    if tf.reduce_mean(input_data) > 0:
      return input_data
    else:
      return input_data // 2


model = CustomModel()

model(tf.constant([-2, -4]))

## Side effects

Tensors defined out of scope, can be manipulated within a function. Just like in eager mode, you can use operations with side effects, like `tf.assign` or `tf.print` normally inside `tf.function`, and it will insert the necessary control dependencies to ensure they execute in order.

In [0]:
v = tf.Variable(5)
print(v.numpy())

@tf.function
def find_next_odd():
  v.assign(v + 1)
  if tf.equal(v % 2, 0):
    v.assign(v + 1)


find_next_odd()
print(v.numpy())

## Example: training a simple model

### Download data

In [0]:
def prepare_mnist_features_and_labels(x, y):
  x = tf.cast(x, tf.float32) / 255.0
  y = tf.cast(y, tf.int64)
  return x, y

def mnist_dataset():
  (x, y), _ = tf.keras.datasets.mnist.load_data()
  ds = tf.data.Dataset.from_tensor_slices((x, y))
  ds = ds.map(prepare_mnist_features_and_labels)
  ds = ds.take(20000).shuffle(20000).batch(100)
  return ds

train_dataset = mnist_dataset()

### Define the model

In [0]:
model = tf.keras.Sequential((
    tf.keras.layers.Reshape(target_shape=(28 * 28,), input_shape=(28, 28)),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(10)))
model.build()
optimizer = tf.keras.optimizers.Adam()

### Define the training loop

For some models, that consist of multiple networks, such as GANs, it can be crucial to wrap the training step with @tf.function.

In [0]:
compute_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

compute_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

@tf.function
def train_one_step(model, optimizer, x, y):
  with tf.GradientTape() as tape:
    logits = model(x)
    loss = compute_loss(y, logits)

  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  compute_accuracy(y, logits)
  return loss


def train(model, optimizer):
  train_ds = mnist_dataset()
  step = 0
  loss = 0.0
  accuracy = 0.0
  for x, y in train_ds:
    step += 1
    loss = train_one_step(model, optimizer, x, y)
    if tf.equal(step % 10, 0):
      tf.print('Step', step, ': loss', loss, '; accuracy', compute_accuracy.result())
  return step, loss, accuracy

step, loss, accuracy = train(model, optimizer)
print('Final step', step, ': loss', loss, '; accuracy', compute_accuracy.result())

## Batching

In real applications batching is essential for performance. The best code to convert to AutoGraph is code where the control flow is decided at the _batch_ level. If making decisions at the individual _example_ level, try to use batch APIs to maintain performance.

For example, if you have the following code in Python:


In [0]:
def square_if_positive(x):
  return [i ** 2 if i > 0 else i for i in x]


print(square_if_positive(range(-5, 5)))
print(timeit.timeit(lambda: square_if_positive(range(-5, 5)), number=10))

You may be tempted to write it in TensorFlow as such (and this would work!):


In [0]:
@tf.function
def square_if_positive_naive(x):
  result = tf.TensorArray(tf.int32, size=x.shape[0])
  for i in tf.range(x.shape[0]):
    if x[i] > 0:
      result = result.write(i, x[i] ** 2)
    else:
      result = result.write(i, x[i])
  return result.stack()


print(square_if_positive_naive(tf.range(-5, 5)))
print(timeit.timeit(lambda: square_if_positive_naive(tf.range(-5, 5)), number=10))

But in this case, it turns out you can write the following:


In [0]:
def square_if_positive_vectorized(x):
  return tf.where(x > 0, x ** 2, x)


print(square_if_positive_vectorized(tf.range(-5, 5)))
print(timeit.timeit(lambda: square_if_positive_vectorized(tf.range(-5, 5)), number=10))

For more information have a look at the tutorial on [performance with tf.function](https://www.tensorflow.org/guide/function?hl=lt)